In [ ]:
import folium
import geopandas as gpd
import rasterio
import numpy as np
from PIL import Image
from rasterio.warp import transform_bounds
from folium.raster_layers import ImageOverlay
import os
from download_tools import warp_to_wgs84


# === Setup ===
geojson_path = "easternshore.geojson"
orig_tif_path = "sample_data/landsat5_eastern_shore_1984-03-01_1984-03-31.tif"
tif_path = warp_to_wgs84(orig_tif_path)
output_dir = "sample_data"

# === Load GeoJSON and center map ===
gdf = gpd.read_file(geojson_path)
bbox = gdf.total_bounds
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2

# === Read TIFF and add each band ===
with rasterio.open(tif_path) as src:
    # Exact raster edges – avoids half‑pixel offset
    left, bottom, right, top = src.bounds
    bounds_latlon = transform_bounds(
        src.crs, "EPSG:4326",
        left, bottom, right, top,
        densify_pts=21  # keeps curvature accurate
    )
    bounds = [
        [bounds_latlon[1], bounds_latlon[0]],
        [bounds_latlon[3], bounds_latlon[2]],
    ]

    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=8,
        width="60%",
        height="400px",
    )

    for band_number in range(1, src.count + 1):
        img_filename = os.path.join(output_dir, f"band{band_number}_overlay.png")

        if os.path.exists(img_filename):
            print(f"Found existing image: {img_filename} — skipping generation.")
        else:
            arr = src.read(band_number).astype(np.float32)
            arr = np.where(np.isnan(arr), 0, arr)
            mask = arr > 0

            vmin, vmax = (0, 1)
            if np.any(mask):
                vmin, vmax = np.percentile(arr[mask], (2, 98))

            # Normalise and build RGB image
            norm_arr = np.clip((arr - vmin) / (vmax - vmin + 1e-6), 0, 1)
            rgb_arr = np.stack([norm_arr] * 3, axis=-1)
            img = Image.fromarray((rgb_arr * 255).astype(np.uint8))
            img.save(img_filename)

        # Add overlay to map
        ImageOverlay(
            name=f"Band {band_number}",
            image=img_filename,
            bounds=bounds,
            opacity=0.6,
            interactive=True,
            cross_origin=False,
            zindex=band_number,
        ).add_to(m)

# === Layer toggle + display ===
folium.LayerControl().add_to(m)

# Display map in a Jupyter notebook or save it
m  # in a notebook this will render the interactive map

In [ ]:
from download_tools import compute_ndwi
import geopandas as gpd
import folium
import rasterio
import numpy as np
from PIL import Image
from rasterio.warp import transform_bounds
from folium.raster_layers import ImageOverlay
import os

# === NDWI Computation ===
ndwi_tif = "sample_data/ndwi_mask.tif"
ndwi_png = "sample_data/ndwi_mask_overlay.png"
tif_path = "sample_data/landsat5_eastern_shore_1984-03-01_1984-03-31.tif"

ndwi_mask = compute_ndwi(
    path=tif_path,
    mission="landsat-5",
    out_path=ndwi_tif,
    display=True
)

In [ ]:
import folium
import geopandas as gpd
import rasterio
import numpy as np
from PIL import Image
from rasterio.warp import transform_bounds
from folium.raster_layers import ImageOverlay
import os

# === Setup ===
geojson_path = "easternshore.geojson"
tif_path = "sample_data/ndwi_mask.tif"
output_dir = "sample_data"

# === Load GeoJSON and center map ===
gdf = gpd.read_file(geojson_path)
bbox = gdf.total_bounds
center_lat = (bbox[1] + bbox[3]) / 2
center_lon = (bbox[0] + bbox[2]) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=8, width="60%", height="400px")

# === Read TIFF and add each band ===
with rasterio.open(tif_path) as src:
    transform = src.transform
    crs = src.crs
    height, width = src.height, src.width

    # Compute image bounds
    left, top = transform * (0, 0)
    right, bottom = transform * (width, height)
    bounds_latlon = transform_bounds(crs, "EPSG:4326", left, bottom, right, top)
    bounds = [[bounds_latlon[1], bounds_latlon[0]], [bounds_latlon[3], bounds_latlon[2]]]
    center_image_lat = (bounds[0][0] + bounds[1][0]) / 2
    center_image_lon = (bounds[0][1] + bounds[1][1]) / 2

    for band_number in range(1, src.count + 1):
        img_filename = os.path.join(output_dir, "water_mask_overlay.png")

        arr = src.read(band_number)
        arr = np.nan_to_num(arr, nan=0)
        
        # If the image is binary (e.g. NDWI water mask), scale directly
        unique_vals = np.unique(arr)
        print("Unique values in band:", unique_vals)
        
        if np.array_equal(unique_vals, [0, 1]) or np.array_equal(unique_vals, [0]) or np.array_equal(unique_vals, [1]):
            # Binary mask → scale directly
            rgb_arr = np.stack([arr]*3, axis=-1)
            img = Image.fromarray((rgb_arr * 255).astype(np.uint8))
        else:
            # Use percentiles for normalization
            valid = arr[np.isfinite(arr)]
            if valid.size > 0:
                vmin, vmax = np.percentile(valid, (2, 98))
            else:
                vmin, vmax = 0, 1
        
            norm_arr = np.clip((arr - vmin) / (vmax - vmin), 0, 1)
            rgb_arr = np.stack([norm_arr]*3, axis=-1)
            img = Image.fromarray((rgb_arr * 255).astype(np.uint8))
        
        img.save(img_filename)


        ImageOverlay(
            name="NDWI Grayscale",
            image=os.path.abspath(img_filename),
            bounds=bounds,
            opacity=0.8,
            interactive=True,
            cross_origin=False,
            zindex=105,
        ).add_to(m)

# Add layer toggle + display
folium.LayerControl().add_to(m)
m


In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import os

import geopandas as gpd

from salinity_tools import (
    load_salinity_truth,
    extract_salinity_features_from_mosaic
)
from download_tools import (
    get_mission,
    find_satellite_coverage,
    download_matching_images
)

truth_path = "data/salinity_labels/codc_salinity_profiles.csv"
salinity_df = load_salinity_truth(truth_path).head(50000)

## 🛰️ Check Satellite Coverage for Salinity Measurements

covered_df = find_satellite_coverage(salinity_df)
overlapped_df = covered_df[covered_df['covered_by'].apply(lambda x: len(x) > 0)]

matched_df = download_matching_images(overlapped_df.head(1))  # Just the first row for demo
matched_df.to_csv("data/ground_truth_matches.csv", index=False)


# Load TIFF paths from the first row in your DataFrame
sample_paths = matched_df["downloaded_files"].iloc[0]
print("Sample paths:", sample_paths)

# First, find the maximum number of bands among the files
max_bands = 0
band_data = []

# Read all bands from each file
for path in sample_paths:
    with rasterio.open(path) as src:
        bands = [src.read(b) for b in range(1, src.count + 1)]
        band_data.append((os.path.basename(path), bands))
        max_bands = max(max_bands, src.count)

# Set up the grid: one row per image, one column per band
n_images = len(band_data)
fig, axes = plt.subplots(n_images, max_bands, figsize=(4 * max_bands, 4 * n_images))

# If only one row/column, make axes always 2D
if n_images == 1:
    axes = np.expand_dims(axes, axis=0)
if max_bands == 1:
    axes = np.expand_dims(axes, axis=1)

# Plot each band in the grid
for row_idx, (filename, bands) in enumerate(band_data):
    for col_idx in range(max_bands):
        ax = axes[row_idx, col_idx]
        if col_idx < len(bands):
            band = bands[col_idx]
            ax.imshow(band, cmap="gray")
            ax.set_title(f"Band {col_idx+1}")
        else:
            ax.set_visible(False)  # Hide unused subplot
        ax.axis("off")
    axes[row_idx, 0].set_ylabel(filename, fontsize=12, rotation=0, labelpad=60, va='center')

plt.tight_layout()
plt.show()



In [ ]:
test_mission = get_mission(matched_df["covered_by"][0][0])  # Get first mission listed
base, _ = os.path.splitext(sample_paths[0])
feature_path = f"{base}_features.tif"
mask_path = f"{base}_mask.tif"

X = extract_salinity_features_from_mosaic(
    sample_paths[0],
    test_mission["band_index"],
    feature_path,
    mask_path
)

print("Extracted Feature Shape:", X.shape)

